# Encontro Python Sergipe 8

## Introdução ao Processamento de Linguagem Natural com Python

### Similaridade entre frases

    Neste notebook vou mostrar como construir um algoritmo que analisa o quanto semelhante são duas frases, apenas para fins de estudo. Está não é uma solução completa, mas te dar uma ideia geral de como pode ser feito esse tipo de comparação apenas usando o conhecimento de álgebra linear.

In [1]:
import re
import string
from math import sqrt

    Para esta analise de similaridade usei algumas frases curtas que foram escolhidas de maneira aleatória.

In [2]:
corpus = [
    'O importante não é vencer todos os dias, mas lutar sempre.',
    'Maior que a tristeza de não haver vencido é a vergonha de não ter lutado!',
    'É melhor conquistar a si mesmo do que vencer mil batalhas.',
    'Oi eu sou o goku!',
    'Oi eu não sou o goku!!!',
    'Oi seu nome é goku(?)'
]

In [3]:
def clear_text(text):
    """
    Limpar os textos, eliminando pontuações e converter 
    todo o texto com letras minusculas.
    """
    pattern = "[{}]".format(string.punctuation)
    text = [word.lower() for word in text]
    text = [[re.sub(pattern, "", word) for word in words.split()] for words in text]
    text = [[word for word in words if len(word)>1] for words in text]    
    text = [' '.join(words) for words in text]
    return text

In [4]:
corpus_clear = clear_text(corpus)

print('-- Antes --')
[print(frase) for frase in corpus]
print()
print('-- Depois --')
[print(frase) for frase in corpus_clear]

-- Antes --
O importante não é vencer todos os dias, mas lutar sempre.
Maior que a tristeza de não haver vencido é a vergonha de não ter lutado!
É melhor conquistar a si mesmo do que vencer mil batalhas.
Oi eu sou o goku!
Oi eu não sou o goku!!!
Oi seu nome é goku(?)

-- Depois --
importante não vencer todos os dias mas lutar sempre
maior que tristeza de não haver vencido vergonha de não ter lutado
melhor conquistar si mesmo do que vencer mil batalhas
oi eu sou goku
oi eu não sou goku
oi seu nome goku


[None, None, None, None, None, None]

    Em seguida vamos construir um vetor com todas as palavras do corpus, após ser feita a limpeza do mesmo.

In [5]:
def text_all(text):
    """
    Armazena em um vetor todas as palavras dos textos sem repetições.
    """
    text_set = set()
    for w in [words.split() for words in text]:
        text_set.update(w)
    return list(text_set) 

In [6]:
vocabulary = text_all(corpus_clear)
print(vocabulary)

['do', 'sempre', 'batalhas', 'mesmo', 'que', 'oi', 'haver', 'nome', 'os', 'todos', 'si', 'melhor', 'seu', 'tristeza', 'sou', 'vencido', 'mas', 'vencer', 'de', 'ter', 'não', 'lutar', 'goku', 'mil', 'eu', 'lutado', 'dias', 'vergonha', 'importante', 'maior', 'conquistar']


In [7]:
def fit_transform(text, words=vocabulary):
    """
    Converte o texto em um vetor, onde compara se cada 
    palavra obtida no vetor de 
    todas as palavras contém ou não em cada texto. 
    Insere 1 se sim e 0 se não.
    """
    
    return [int(word in text.split()) for word in words]

features = list(map(fit_transform, corpus_clear))

for feature in features:
    print(feature)

[0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0]
[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0]
[1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]


    Para determinar o quanto similar são os comentários, é necessário utilizar algum procedimento (formula) para medir esta similaridade. Uma delas é a métrica chamada similaridade do cosseno. Quem já cursou álgebra linear deve saber que o produto vetorial de dois vetores é igual ao modulo de cada vetor vezes o cosseno do ângulo formado entre eles. E é justamente o cosseno que utilizaremos para determinar o quanto similar são os comentários.

In [8]:
def dot(v, w):
    return sum([vi*wi for vi, wi in zip(v,w)])

def cosine_similarity(v, w):
    return dot(v, w)/sqrt(dot(v, v)*dot(w, w))    

In [9]:
def text_simillarities(id_text, features=features, 
                       text=corpus, n_text=3):
    """
    Dado o texto a ser analisado, a função retorna em 
    ordem descrecente quais os demais textos são
    similares ao analisado. A função retorna matriz 
    de 2 por n_text, onde a primeira e a segunda coluna
    refere-se ao texto analisado e a similaridade do 
    texto analisado, respectivamente.
    """
    simillarity = [[cosine_similarity(features[id_text], feature), int(i)] \
                   for i, feature in enumerate(features)]
    
    simillarity = list(sorted(simillarity, 
                              key=lambda sim: sim[0], 
                              reverse=True))    
    
    return [[text[indice], sim] for sim, indice in simillarity[1:]][:n_text]


In [10]:
print('Texto analisado -> ',corpus[3], '\n')
for t, s in text_simillarities(3):
    print('Texto: {} | Similaridade: {}'.format(t, round(s, 2)))

Texto analisado ->  Oi eu sou o goku! 

Texto: Oi eu não sou o goku!!! | Similaridade: 0.89
Texto: Oi seu nome é goku(?) | Similaridade: 0.5
Texto: O importante não é vencer todos os dias, mas lutar sempre. | Similaridade: 0.0
